In [16]:
# Installing packages
#!pip install pyreadr

# Importing the libraries
import os
import pandas as pd
import pyreadr


# Set the working directory
new_working_directory = r"G:\.shortcut-targets-by-id\1BICzskIUWPIDEYveSj80Ks6gQhVX7EkP\Project Code"
os.chdir(new_working_directory)

wd = os.getcwd()

In [12]:
# Reading the dataset with MTA yearly totals
mta_yearly_ridership = pd.read_excel(f"{wd}/RawData/MTARidership/2023 Subway Tables.xlsx",
                                     sheet_name="Annual Total",
                                     skiprows=1)
# Correcting column names
mta_yearly_ridership.columns = ["station", "closure_id", "borough", 
                                "2018", "2019", "2020", "2021", "2022", "2023", 
                                "22_23_diff", "22_23_diff_percent", "2023_rank"]

# Filtering out unwanted rows
unwanted_rows = ["The Bronx", "Brooklyn", "Manhattan", "Queens", "Bronx", 
                 "Systemwide Adjustment", "System Total", None, 
                 "* MTA New York City Transit Temporary Station Closings, 2018-2023"]

mta_yearly_ridership = mta_yearly_ridership[~mta_yearly_ridership['station'].isin(unwanted_rows)]

# Renaming columns
mta_yearly_ridership = mta_yearly_ridership.rename(columns={
    "2018": "mta2018",
    "2019": "mta2019",
    "2020": "mta2020",
    "2021": "mta2021",
    "2022": "mta2022",
    "2023": "mta2023"
})

# Changing the name of the stations
mta_yearly_ridership['station'] = mta_yearly_ridership['station'].str.upper().str.strip()
mta_yearly_ridership['station'] = mta_yearly_ridership['station'].str.replace(",", "")
mta_yearly_ridership['station'] = mta_yearly_ridership['station'].str.replace(r"\(BC\)", "(CB)")
mta_yearly_ridership['station'] = mta_yearly_ridership['station'].str.replace(r"\(ABCD\)", "(ACBD)")
mta_yearly_ridership['station'] = mta_yearly_ridership['station'].str.replace(r"\(JM\)", "(MJ)")
mta_yearly_ridership['station'] = mta_yearly_ridership['station'].str.replace(r"\(JMZ\)", "(MJZ)")

station_replacements = {
    "39 AV (NW)": "39 AV-DUTCH KILLS (NW)",
    "47-50 STS-ROCKEFELLER CENTER (BDFM)": "47-50 STS-ROCKEFELLER CTR (BDFM)",
    "5 AV-53 ST (EM)": "5 AV/53 ST (EM)",
    "5 AV-59 ST (NRW)": "5 AV/59 ST (NRW)",
    "7 AV (BDE)": "7 AV (EBD)",
    "8 ST-NEW YORK UNIVERSITY (RW)": "8 ST-NYU (RW)",
    "86 ST. (1)": "86 ST (1)",
    "86 ST. (456)": "86 ST (456)",
    "86 ST. (CB)": "86 ST (CB)",
    "86 ST. (N)": "86 ST (N)",
    "86 ST. (R)": "86 ST (R)",
    "AQUEDUCT-NORTH CONDUIT AV (A)": "AQUEDUCT-N CONDUIT AV (A)",
    "ASTOR PLACE (6)": "ASTOR PL (6)",
    "BEACH 67 ST-ARVERNE BY THE SEA (A)": "BEACH 67 ST (A)",
    "BRIARWOOD-VAN WYCK BLVD (EF)": "BRIARWOOD (EF)",
    "CATHEDRAL PKWY-110 ST (1)": "CATHEDRAL PKWY (110 ST) (1)",
    "CATHEDRAL PKWY-110 ST (CB)": "CATHEDRAL PKWY (110 ST) (CB)",
    "CENTRAL PARK NORTH-110 ST (23)": "CENTRAL PARK NORTH (110 ST) (23)",
    "CHAMBERS ST (AC)/WTC (E)/PARK PLACE (23)/CORTLANDT (RW)": "CHAMBERS ST (AC)/WTC (E)/PARK PL (23)/CORTLANDT (RW)",
    "CROWN HEIGHTS-UTICA AV (34)": "CROWN HTS-UTICA AV (34)",
    "DELANCEY ST (F)/ESSEX ST (MJZ)": "DELANCEY ST (F)/ESSEX ST (JMZ)",
    "EAST 143 ST-ST MARY'S ST (6)": "E 143 ST-ST MARY'S ST (6)",
    "EAST 149 ST (6)": "E 149 ST (6)",
    "EAST 180 ST (25)": "E 180 ST (25)",
    "JAMAICA CENTER-PARSONS-ARCHER (EJZ)": "JAMAICA CENTER-PARSONS/ARCHER (EJZ)",
    "KEW GARDENS-UNION TURNPIKE (EF)": "KEW GARDENS-UNION TPKE (EF)",
    "LEXINGTON AV-63 ST (FQ)": "LEXINGTON AV/63 ST (FQ)",
    "LEXINGTON AV (NQR)/59 ST (456)": "LEXINGTON AV (NRW)/59 ST (456)",
    "NEWKIRK AV (25)": "NEWKIRK AV-LITTLE HAITI (25)",
    "PRESIDENT ST (25)": "PRESIDENT ST-MEDGAR EVERS COLLEGE (25)",
    "QUEENSBORO PLAZA (NW7)": "QUEENSBORO PLAZA (7NW)",
    "ST. LAWRENCE AVE. (6)": "ST LAWRENCE AV (6)",
    "STEINWAY ST. (MR)": "STEINWAY ST (MR)",
    "STERLING ST. (25)": "STERLING ST (25)",
    "TIMES SQ-42 ST (NQRSW1237)/42 ST (ACE)/BRYANT PK (BDFM)/5 AV (7)": "TIMES SQ-42 ST (NQRWS1237)/42 ST (ACE)",
    "WEST 4 ST-WASHINGTON SQ (ABCDEFM)": "W 4 ST-WASH SQ (ACEBDFM)",
    "WEST 8 ST-NEW YORK AQUARIUM (FQ)": "W 8 ST-NY AQUARIUM (FQ)",
    "WEST FARMS SQ-EAST TREMONT AV (25)": "WEST FARMS SQ-E TREMONT AV (25)",
    "WESTCHESTER SQ-EAST TREMONT AV (6)": "WESTCHESTER SQ-E TREMONT AV (6)",
    "WOODSIDE-61 ST (7)": "61 ST-WOODSIDE (7)"
}

mta_yearly_ridership['station'] = mta_yearly_ridership['station'].replace(station_replacements, regex=True)

In [33]:
# Loading the quarterly ridership data we got from daily subway entries
result = pyreadr.read_r(f"{wd}/CleanedData/05c_ridership_quarterly.rds")

# Extracting the dataframe
ridership_data = result[None] 

# Getting annual totals
ridership_data = ridership_data.groupby(['year', 'station_complex_id', 'display_name'], as_index=False)['quarterly_entries'].sum()
ridership_data = ridership_data.rename(columns={'quarterly_entries': 'ridership'})

# Changing to wide format
ridership_data_wide = ridership_data.pivot(index=['station_complex_id', 'display_name'], columns='year', values='ridership').reset_index()

# Changing column names
ridership_data_wide = ridership_data_wide.rename(columns={
    2017: "r2017",
    2018: "r2018",
    2019: "r2019",
    2020: "r2020",
    2021: "r2021",
    2022: "r2022",
    2023: "r2023"
})

# Joining station names
station_id = ridership_data_wide[['station_complex_id', 'display_name']].drop_duplicates().rename(columns={'display_name': 'station'})

# Confirming all of them match
unmatched = mta_yearly_ridership.merge(station_id, on='station', how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only', ['station']]

# Joining both data frames, and calculating % difference 
comparison = mta_yearly_ridership.merge(ridership_data_wide, left_on='station', right_on='display_name', how='left')

In [34]:
for year in range(2018, 2024):
    comparison[f'pct_diff_{year}'] = (comparison[f'r{year}'] - comparison[f'mta{year}']) / comparison[f'mta{year}']

# Summarizing the comparison
summary = comparison.describe()

# Display the summary
print(summary)

       closure_id       mta2018       mta2019       mta2020       mta2021  \
count   46.000000  4.230000e+02  4.230000e+02  4.230000e+02  4.230000e+02   
mean    23.913043  3.971804e+06  4.013681e+06  1.511917e+06  1.796635e+06   
std     13.969532  6.206464e+06  6.238699e+06  1.991494e+06  2.331261e+06   
min      1.000000  8.224800e+04  8.843900e+04  3.583696e+04  3.998100e+04   
25%     12.250000  1.277613e+06  1.341886e+06  5.834105e+05  6.728755e+05   
50%     23.500000  2.221270e+06  2.211198e+06  9.576060e+05  1.122527e+06   
75%     35.750000  4.445027e+06  4.495072e+06  1.711707e+06  2.040925e+06   
max     48.000000  8.111678e+07  8.223400e+07  2.574663e+07  2.950756e+07   

            mta2022       mta2023    22_23_diff  22_23_diff_percent  \
count  4.230000e+02  4.230000e+02  4.230000e+02          423.000000   
mean   2.395805e+06  2.723400e+06  3.275950e+05            0.095857   
std    3.437142e+06  4.149478e+06  7.547335e+05            0.104132   
min    4.963100e+04  5